In [1]:
import os
print('os.getcwd()', os.getcwd())
import sys
sys.path.insert(1, '../')
print(sys.version)
import time

#plotting tools
from matplotlib import pyplot as plt 
from tqdm.notebook import tqdm as tqdm

#torch libs
import torch
print('torch.__version__', torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe_device = 0 if torch.cuda.is_available() else -1
print(pipe_device, device)

#huggingface transformers
import transformers
print('transformers.__version__',transformers.__version__)
from transformers import AutoTokenizer, pipeline
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2PreTrainedModel


#curious
from curious.models import GPT2HeadWithValueModel
from curious.rl import PPOTrainer
from curious.utils import LengthSampler, collater, respond_to_batch, generate_text

#jupyter stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

os.getcwd() /home/carson/Desktop/Projects/language_reinforce/notebooks
3.7.5 (default, Dec  9 2021, 17:04:37) 
[GCC 8.4.0]
torch.__version__ 1.12.1+cu102
0 cuda
transformers.__version__ 4.22.2


In [2]:
#cache_folder = '/Users/carson/projects/modelstates' 
cache_folder = '/home/carson/Desktop/Projects/modelstates'
model_name = 'distilgpt2' #'gpt2' # 'gpt2-xl' # 'gpt2-large' #
cache_dir = os.path.join(cache_folder, model_name)
cache_dir

'/home/carson/Desktop/Projects/modelstates/distilgpt2'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    pad_token='<|endoftext|>',
    padding_side = 'left',
)

model = GPT2LMHeadModel.from_pretrained(
    model_name,
    cache_dir = cache_dir, 
    pad_token_id=tokenizer.eos_token_id,
)

model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

```
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (i): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
```

In [4]:
print(model.lm_head.weight.shape)
model.lm_head.weight

torch.Size([50257, 768])


Parameter containing:
tensor([[-0.1445, -0.0455,  0.0042,  ..., -0.1523,  0.0184,  0.0991],
        [ 0.0573, -0.0722,  0.0234,  ...,  0.0603, -0.0042,  0.0478],
        [-0.1106,  0.0386,  0.1948,  ...,  0.0421, -0.1141, -0.1455],
        ...,
        [-0.0710, -0.0173,  0.0176,  ...,  0.0834,  0.1340, -0.0746],
        [ 0.1993,  0.0201,  0.0151,  ..., -0.0829,  0.0750, -0.0294],
        [ 0.0342,  0.0640,  0.0305,  ...,  0.0291,  0.0942,  0.0639]],
       requires_grad=True)

In [5]:
gpt2_model = GPT2HeadWithValueModel.from_pretrained(
    model_name,
    cache_dir=cache_dir
)
print(gpt2_model.lm_head.weight.shape)
gpt2_model.lm_head.weight

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['transformer.h.3.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'v_head.summary.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'v_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([50257, 768])


Parameter containing:
tensor([[-0.1445, -0.0455,  0.0042,  ..., -0.1523,  0.0184,  0.0991],
        [ 0.0573, -0.0722,  0.0234,  ...,  0.0603, -0.0042,  0.0478],
        [-0.1106,  0.0386,  0.1948,  ...,  0.0421, -0.1141, -0.1455],
        ...,
        [-0.0710, -0.0173,  0.0176,  ...,  0.0834,  0.1340, -0.0746],
        [ 0.1993,  0.0201,  0.0151,  ..., -0.0829,  0.0750, -0.0294],
        [ 0.0342,  0.0640,  0.0305,  ...,  0.0291,  0.0942,  0.0639]],
       requires_grad=True)

In [6]:
generate_text(gpt2_model, tokenizer, prompt = "the first rule of robotics is")

['the first rule of robotics is that you never set a rule regarding speed, but you just want to look at']